<a href="https://colab.research.google.com/github/iskra3138/colab_seminar/blob/master/colab%EC%97%90%EC%84%9C_KorQuad_%ED%95%99%EC%8A%B5%ED%95%98%EA%B8%B02_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Colab에서 KorQuAD 학습하기 2
- TPU instance와 Google Drive, Google Cloud 활용
- Google Cloud 계정이 있어야만 작동하는 코드입니다.

## 구글 클라우드 마운트

In [1]:
# Google Cloud 계정 인증
from google.colab import auth
auth.authenticate_user()

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [2]:
# project ID 확인
!gcloud projects list

PROJECT_ID             NAME              PROJECT_NUMBER
optimal-bivouac-98502  test              40457369823
precise-tenure-263301  My First Project  20496313163


To take a quick anonymous survey, run:
  $ gcloud survey



- PROJECT에는 위 셀 output을 참고해서 작성
- PRE_BUCKET에는 pre_trained model 이 저장될 storage의 이름 "gs://name"
- OUT_BUCKET에는 output을 저장할 storage의 이름 "gs://name"

In [0]:
PROJECT = "precise-tenure-263301" #@param {type:"string"}
PRE_BUCKET = "gs://iskra3138_pretrained_files" #@param {type:"string"}
OUT_BUCKET = "gs://iskra3138_temp" #@param {type:"string"}

In [4]:
## pretrained files이 저장될 공간(버킷)과 output이 저장될 공간(버킷) 생성

!gsutil mb {PRE_BUCKET}
!gsutil mb {OUT_BUCKET}

Creating gs://iskra3138_pretrained_files/...
You are attempting to perform an operation that requires a project id, with none configured. Please re-run gsutil config and make sure to follow the instructions for finding and entering your default project id.
Creating gs://iskra3138_temp/...
You are attempting to perform an operation that requires a project id, with none configured. Please re-run gsutil config and make sure to follow the instructions for finding and entering your default project id.


## 구글 드라이브 마운트

In [5]:
from google.colab import drive
drive.mount('/gdrive') # 'gdrive'라는 이름으로 Google Drive를 Mount하겠다!

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


- Google Drive의 최상위 Directory는 '위에서 선언한 이름/My Drive'가 됨

### 실험 공간 생성

In [0]:
## 실험 공간 선언
path='/gdrive/My Drive/KorQuAD'

In [0]:
# 실험을 위한 Directory 생성
import os
if not os.path.exists(path):
  os.makedirs(path)

In [8]:
# 실험 Directory로 이동 및 확인
os.chdir(path)
os.getcwd()

'/gdrive/My Drive/KorQuAD'

### 필요한 파일 다운 받기
KorQuAD 학습하기1-GPU에서 파일들을 이미 받았다면 생략

In [9]:
# pre-train된 모델 다운받기
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip

--2020-01-09 17:11:45--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.111.128, 2607:f8b0:4001:c1b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.111.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M  48.6MB/s    in 10s     

2020-01-09 17:11:56 (62.1 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]



In [10]:
# 압축 풀고 압축파일 삭제
!unzip multi_cased_L-12_H-768_A-12.zip -d pretrained_files
!rm multi_cased_L-12_H-768_A-12.zip

Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: pretrained_files/multi_cased_L-12_H-768_A-12/
  inflating: pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: pretrained_files/multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: pretrained_files/multi_cased_L-12_H-768_A-12/bert_config.json  


In [11]:
# KorQuAD Data 다운받기

!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
!wget https://korquad.github.io/dataset/evaluate-v1.0.py

--2020-01-09 17:12:14--  https://korquad.github.io/dataset/KorQuAD_v1.0_train.json
Resolving korquad.github.io (korquad.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 38527475 (37M) [application/json]
Saving to: ‘KorQuAD_v1.0_train.json’

KorQuAD_v1.0_train. 100%[===================>]  36.74M  78.1MB/s    in 0.5s    

2020-01-09 17:12:15 (78.1 MB/s) - ‘KorQuAD_v1.0_train.json’ saved [38527475/38527475]

--2020-01-09 17:12:16--  https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json
Resolving korquad.github.io (korquad.github.io)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to korquad.github.io (korquad.github.io)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3881058 (3.7M) [application/json]
Saving to: ‘KorQuAD_v1.0_dev.json’

KorQuAD_v1.0_dev.js 100%[=====

In [12]:
# 학습에 필요한 코드 다운받기
!git clone https://github.com/google-research/bert.git ./bert_files

Cloning into './bert_files'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 287.79 KiB | 3.74 MiB/s, done.
Resolving deltas: 100% (184/184), done.


### TPU학습을 위해 필요한 파일들을 구글드라이브에서 구글클라우드로 옮기기

In [0]:
import os
if not os.getcwd()==path:
  os.chdir(path)
  os.getcwd()

In [14]:
pre_drive='{}/pretrained_files'.format(path)
print (pre_drive)

/gdrive/My Drive/KorQuAD/pretrained_files


In [15]:
## 로컬 -> 버킷으로 파일 복사(=업로드)
!gsutil cp -r '{pre_drive}' '{PRE_BUCKET}'

Copying file:///gdrive/My Drive/KorQuAD/pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta [Content-Type=application/octet-stream]...
Copying file:///gdrive/My Drive/KorQuAD/pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001 [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying fil

### TPU 주소 확인하고 학습 실행
- 재실행시 TPU 주소가 변경됨

In [16]:
import tensorflow as tf
import numpy as np
import os

try:
  device_name = os.environ['COLAB_TPU_ADDR']
  TPU_ADDRESS = 'grpc://' + device_name
  print('Found TPU at: {}'.format(TPU_ADDRESS))

except KeyError:
  print('TPU not found')

Found TPU at: grpc://10.121.229.98:8470


In [17]:
!gsutil ls {PRE_BUCKET}/pretrained_files/

gs://iskra3138_pretrained_files/pretrained_files/multi_cased_L-12_H-768_A-12/


In [18]:
## 각 파일들의 경로 잡아주고 TPU 학습
!nohup python ./bert_files/run_squad.py \
  --vocab_file={PRE_BUCKET}/pretrained_files/multi_cased_L-12_H-768_A-12/vocab.txt \
  --bert_config_file={PRE_BUCKET}/pretrained_files/multi_cased_L-12_H-768_A-12/bert_config.json \
  --init_checkpoint={PRE_BUCKET}/pretrained_files/multi_cased_L-12_H-768_A-12/bert_model.ckpt \
  --do_train=True \
  --train_file=./KorQuAD_v1.0_train.json \
  --do_predict=True \
  --predict_file=./KorQuAD_v1.0_dev.json \
  --train_batch_size=16 \
  --learning_rate=3e-5 \
  --num_train_epochs=1.0 \
  --max_seq_length=256 \
  --doc_stride=128 \
  --output_dir={OUT_BUCKET} \
  --use_tpu=True \
  --tpu_name=$TPU_ADDRESS \
  --do_lower_case=False \
  > log_tpu.txt &

nohup: redirecting stderr to stdout


### log monitoring

In [0]:
!tail -f log_tpu.txt

I0109 17:20:47.309856 139949708806016 session_support.py:82] Creating heartbeat manager for ['/job:worker/replica:0/task:0/device:CPU:0']
I0109 17:20:47.311743 139949708806016 session_support.py:105] Configuring worker heartbeat: shutdown_mode: WAIT_FOR_COORDINATOR

I0109 17:20:47.316382 139949708806016 tpu_estimator.py:567] Init TPU system
I0109 17:20:52.131591 139949708806016 tpu_estimator.py:576] Initialized TPU in 4 seconds
I0109 17:20:52.132433 139947973801728 tpu_estimator.py:521] Starting infeed thread controller.
I0109 17:20:52.133394 139947936610048 tpu_estimator.py:540] Starting outfeed thread controller.
I0109 17:20:52.596939 139949708806016 tpu_estimator.py:600] Enqueue next (1000) batch(es) of data to infeed.
I0109 17:20:52.597553 139949708806016 tpu_estimator.py:604] Dequeue next (1000) batch(es) of data from outfeed.
I0109 17:21:21.437103 139947936610048 tpu_estimator.py:279] Outfeed finished for iteration (0, 0)
I0109 17:22:19.122221 139949708806016 basic_session_run_ho

### Tensorboard 실행

In [20]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2020-01-09 17:18:26--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 34.235.253.190, 52.20.12.96, 54.164.7.157, ...
Connecting to bin.equinox.io (bin.equinox.io)|34.235.253.190|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13773305 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.13M  31.3MB/s    in 0.4s    

2020-01-09 17:18:27 (31.3 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13773305/13773305]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [0]:
LOG_DIR = {OUT_BUCKET}
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [0]:
get_ipython().system_raw('./ngrok http 6006 &')

In [23]:
!curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://1a60c4a3.ngrok.io


### 성능 평가를 위해 필요한 파일들을 구글 클라우드에서 구글드라이브로 다운로드

In [0]:
## 버킷 -> 로컬로 파일 복사(=다운로드)

!mkdir temp_tpu # ouput을 저장할 폴더 생성

!gsutil cp {OUT_BUCKET}/predictions.json ./temp_tpu/predictions.json

In [0]:
# 평가
!python ./evaluate-v1.0.py ./KorQuAD_v1.0_dev.json ./temp_tpu/predictions.json

### 구글클라우드 스토리지에 저장된 파일들 삭제

In [0]:
## 생성된 bucket 삭제
!gsutil rm -r {PRE_BUCKET}
!gsutil rm -r {OUT_BUCKET}

### log 파일을 local로 다운로드

In [0]:
from google.colab import files

files.download('./log_tpu.txt')

## Clean Up
Before running the next exercise, run the following cell to terminate the kernel and free memory resources:

In [0]:
import os, signal
os.kill(os.getpid(), signal.SIGKILL)